In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import glob
import os
import re
import sys
import xarray as xr
import rioxarray as rio
sys.path.append(os.path.abspath("c:\\Users\\m1865\\Desktop\\GIS Thesis Python"))
import mySTD
# Data directory of ARPA data
dd_arpa = "c:\\Users\\m1865\\Desktop\\GIS Thesis Python\\ARPA"
dd_cams = "c:\\Users\\m1865\\Desktop\\GIS Thesis Python\\CAMS"
dd_main = "c:\\Users\\m1865\\Desktop\\GIS Thesis Python"
# Set current working directory
cwd = os.getcwd()
cwd

In [ ]:
myFile_NO2 = []
myFile_O3 = []
myFile_PM25 = []
myFile_PM10 = []
# Filter all the file names of the data from the data folder
for filename in os.listdir(dd_cams):
    match = re.match(r'(.*)2021(.*).nc', filename)
    # print('Check Match')
    if match:
        # print('Matched!')
        if match.group(1) == "NO2":
            myFile_NO2.append(filename)
        elif match.group(1) == "ozone":
            myFile_O3.append(filename)
        elif match.group(1) == "PM25": 
            myFile_PM25.append(filename)
        elif match.group(1) == "PM10":
            myFile_PM10.append(filename)

In [ ]:
myList_NO2 = []
myList_O3 = []
myList_PM10 = []
myList_PM25 = []
for filename in myFile_NO2:
    ds = xr.open_dataset(dd_cams + "\\" + filename)
    newlong = mySTD.Lon360to180(ds)
    newtime = mySTD.TimetoDate64(ds)
    ds = ds.assign_coords(time=newtime,longitude=newlong).sortby("longitude")
    # print(ds.time.dt.month)
    myList_NO2.append(ds)
for filename in myFile_O3:
    ds = xr.open_dataset(dd_cams + "\\" + filename)
    newlong = mySTD.Lon360to180(ds)
    newtime = mySTD.TimetoDate64(ds)
    ds = ds.assign_coords(time=newtime,longitude=newlong).sortby("longitude")
    myList_O3.append(ds)
for filename in myFile_PM10:
    ds = xr.open_dataset(dd_cams + "\\" + filename)
    newlong = mySTD.Lon360to180(ds)
    newtime = mySTD.TimetoDate64(ds)
    ds = ds.assign_coords(time=newtime,longitude=newlong).sortby("longitude")
    myList_PM10.append(ds)
for filename in myFile_PM25:
    ds = xr.open_dataset(dd_cams + "\\" + filename)
    newlong = mySTD.Lon360to180(ds)
    newtime = mySTD.TimetoDate64(ds)
    ds = ds.assign_coords(time=newtime,longitude=newlong).sortby("longitude")
    myList_PM25.append(ds)

In [ ]:
# Check
myList_NO2[1]

In [ ]:
myList_NO2_Mean = []
myList_O3_Mean = []
myList_PM10_Mean = []
myList_PM25_Mean = []
for i in range(len(myList_NO2)):
    ds = myList_NO2[i]
    ds = ds.mean(dim='time')
    ds = ds.mean(dim='level')
    ds = ds.assign_attrs({"month": i+1})
    myList_NO2_Mean.append(ds)
for i in range(len(myList_O3)):
    ds = myList_O3[i]
    ds = ds.mean(dim='time')
    ds = ds.mean(dim='level')
    ds = ds.assign_attrs({"month": i+1})
    myList_O3_Mean.append(ds)
for i in range(len(myList_PM10)):
    ds = myList_PM10[i]
    ds = ds.mean(dim='time')
    ds = ds.mean(dim='level')
    ds = ds.assign_attrs({"month": i+1})
    myList_PM10_Mean.append(ds)
for i in range(len(myList_PM25)):
    ds = myList_PM25[i]
    ds = ds.mean(dim='time')
    ds = ds.mean(dim='level')
    ds = ds.assign_attrs({"month": i+1})
    myList_PM25_Mean.append(ds)

In [ ]:
# Check
myList_PM25_Mean[4]

In [ ]:
lom = gpd.read_file('..\lom.shp')

In [ ]:
myList_NO2_Clipped = []
myList_O3_Clipped = []
myList_PM10_Clipped = []
myList_PM25_Clipped = []
for i in range(len(myList_NO2_Mean)):
    ds = myList_NO2_Mean[i]
    ds.rio.write_crs(4326, inplace=True)
    ds = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_NO2_Clipped.append(ds)
for i in range(len(myList_O3_Mean)):
    ds = myList_O3_Mean[i]
    ds.rio.write_crs(4326, inplace=True)
    ds = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_O3_Clipped.append(ds)
for i in range(len(myList_PM10_Mean)):
    ds = myList_PM10_Mean[i]
    ds.rio.write_crs(4326, inplace=True)
    ds = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_PM10_Clipped.append(ds)
for i in range(len(myList_PM25_Mean)):
    ds = myList_PM25_Mean[i]
    ds.rio.write_crs(4326, inplace=True)
    ds = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_PM25_Clipped.append(ds)


In [ ]:
print(myList_NO2_Clipped[0].mean())
myList_NO2_Clipped[0]

In [ ]:
# Upscale CAMS
myList_NO2_Clipped_Up = []
myList_O3_Clipped_Up = []
myList_PM10_Clipped_Up = []
myList_PM25_Clipped_Up = []
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial
for i in range(len(myList_NO2_Clipped)):
    ds = myList_NO2_Clipped[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.no2_conc,geometry=gpd.points_from_xy(ds.longitude,ds.latitude))
    ds = ds.dropna()
    ds = make_geocube(
        vector_data=ds,
        measurements=['no2_conc'],
        resolution=(-0.01, 0.01),
        rasterize_function=rasterize_points_griddata,
    )
    ds.rio.write_crs(4326, inplace=True)
    upscale_clip = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_NO2_Clipped_Up.append(upscale_clip)
for i in range(len(myList_O3_Clipped)):
    ds = myList_O3_Clipped[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.o3_conc,geometry=gpd.points_from_xy(ds.longitude,ds.latitude))
    ds = ds.dropna()
    ds = make_geocube(
        vector_data=ds,
        measurements=['o3_conc'],
        resolution=(-0.01, 0.01),
        rasterize_function=rasterize_points_griddata,
    )
    ds.rio.write_crs(4326, inplace=True)
    upscale_clip = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_O3_Clipped_Up.append(upscale_clip)
for i in range(len(myList_PM10_Clipped)):
    ds = myList_PM10_Clipped[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.pm10_conc,geometry=gpd.points_from_xy(ds.longitude,ds.latitude))
    ds = ds.dropna()
    ds = make_geocube(
        vector_data=ds,
        measurements=['pm10_conc'],
        resolution=(-0.01, 0.01),
        rasterize_function=rasterize_points_griddata,
    )
    ds.rio.write_crs(4326, inplace=True)
    upscale_clip = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_PM10_Clipped_Up.append(upscale_clip)
for i in range(len(myList_PM25_Clipped)):
    ds = myList_PM25_Clipped[i].to_dataframe().reset_index()
    ds = gpd.GeoDataFrame(ds.pm2p5_conc,geometry=gpd.points_from_xy(ds.longitude,ds.latitude))
    ds = ds.dropna()
    ds = make_geocube(
        vector_data=ds,
        measurements=['pm2p5_conc'],
        resolution=(-0.01, 0.01),
        rasterize_function=rasterize_points_griddata,
    )
    ds.rio.write_crs(4326, inplace=True)
    upscale_clip = ds.rio.clip(lom.geometry, lom.crs, drop=True, invert = False)
    myList_PM25_Clipped_Up.append(upscale_clip)

In [ ]:
# Calculate average, variance, F-Test p value and T-Test p value for each pollutant, and the result for each one will be stored into a pandas dataframe. 
resultdf_NO2 = pd.DataFrame(columns=['Pollutant','Month','Average','Variance','Upscale Average','Upscale Variance','F-Test p Value','T-Test p Value'])
resultdf_O3 = pd.DataFrame(columns=['Pollutant','Month','Average','Variance','Upscale Average','Upscale Variance','F-Test p Value','T-Test p Value'])
resultdf_PM10 = pd.DataFrame(columns=['Pollutant','Month','Average','Variance','Upscale Average','Upscale Variance','F-Test p Value','T-Test p Value'])
resultdf_PM25 = pd.DataFrame(columns=['Pollutant','Month','Average','Variance','Upscale Average','Upscale Variance','F-Test p Value','T-Test p Value'])
resultdf_NO2

In [ ]:
import scipy.stats
from scipy.stats import ttest_ind
def f_test(x, y):
    f = np.var(x, ddof=1)/np.var(y, ddof=1)
    nun = x.size-1
    dun = y.size-1
    p_value = 1-scipy.stats.f.cdf(f, nun, dun)
    return f, p_value

In [ ]:
# NO2
for i in range(12):
    df_Pollutant = 'NO2'
    df_Month = i+1
    df_Average = myList_NO2_Clipped[i].no2_conc.mean().data
    df_Variance = myList_NO2_Clipped[i].no2_conc.std().data
    df_UP_Average = myList_NO2_Clipped_Up[i].no2_conc.mean().data
    df_UP_Variance = myList_NO2_Clipped_Up[i].no2_conc.std().data

    df_cams = myList_NO2_Clipped[i].no2_conc.data
    df_cams = df_cams[~np.isnan(df_cams)]
    df_cams = df_cams.reshape(-1)
    
    df_camsup = myList_NO2_Clipped_Up[i].no2_conc.data
    df_camsup = df_camsup[~np.isnan(df_camsup)]
    df_camsup = df_camsup.reshape(-1)

    df_FTest = f_test(df_cams,df_camsup)[1]
    df_TTest = ttest_ind(df_cams,df_camsup)[1]

    df_data = pd.DataFrame({
        'Pollutant': [df_Pollutant], 
        'Month': [df_Month],
        'Average': [df_Average],
        'Variance': [df_Variance],
        'Upscale Average': [df_UP_Average],
        'Upscale Variance': [df_UP_Variance],
        'F-Test p Value': [df_FTest],
        'T-Test p Value': [df_TTest]
    })
    resultdf_NO2 = pd.concat([resultdf_NO2,df_data])

In [ ]:
resultdf_NO2

In [ ]:
# O3
for i in range(12):
    df_Pollutant = 'O3'
    df_Month = i+1
    df_Average = myList_O3_Clipped[i].o3_conc.mean().data
    df_Variance = myList_O3_Clipped[i].o3_conc.std().data
    df_UP_Average = myList_O3_Clipped_Up[i].o3_conc.mean().data
    df_UP_Variance = myList_O3_Clipped_Up[i].o3_conc.std().data

    df_cams = myList_O3_Clipped[i].o3_conc.data
    df_cams = df_cams[~np.isnan(df_cams)]
    df_cams = df_cams.reshape(-1)
    
    df_camsup = myList_O3_Clipped_Up[i].o3_conc.data
    df_camsup = df_camsup[~np.isnan(df_camsup)]
    df_camsup = df_camsup.reshape(-1)

    df_FTest = f_test(df_cams,df_camsup)[1]
    df_TTest = ttest_ind(df_cams,df_camsup)[1]

    df_data = pd.DataFrame({
        'Pollutant': [df_Pollutant], 
        'Month': [df_Month],
        'Average': [df_Average],
        'Variance': [df_Variance],
        'Upscale Average': [df_UP_Average],
        'Upscale Variance': [df_UP_Variance],
        'F-Test p Value': [df_FTest],
        'T-Test p Value': [df_TTest]
    })
    resultdf_O3 = pd.concat([resultdf_O3,df_data])

In [ ]:
resultdf_O3

In [ ]:
# PM10
for i in range(12):
    df_Pollutant = 'PM10'
    df_Month = i+1
    df_Average = myList_PM10_Clipped[i].pm10_conc.mean().data
    df_Variance = myList_PM10_Clipped[i].pm10_conc.std().data
    df_UP_Average = myList_PM10_Clipped_Up[i].pm10_conc.mean().data
    df_UP_Variance = myList_PM10_Clipped_Up[i].pm10_conc.std().data

    df_cams = myList_PM10_Clipped[i].pm10_conc.data
    df_cams = df_cams[~np.isnan(df_cams)]
    df_cams = df_cams.reshape(-1)
    
    df_camsup = myList_PM10_Clipped_Up[i].pm10_conc.data
    df_camsup = df_camsup[~np.isnan(df_camsup)]
    df_camsup = df_camsup.reshape(-1)

    df_FTest = f_test(df_cams,df_camsup)[1]
    df_TTest = ttest_ind(df_cams,df_camsup)[1]

    df_data = pd.DataFrame({
        'Pollutant': [df_Pollutant], 
        'Month': [df_Month],
        'Average': [df_Average],
        'Variance': [df_Variance],
        'Upscale Average': [df_UP_Average],
        'Upscale Variance': [df_UP_Variance],
        'F-Test p Value': [df_FTest],
        'T-Test p Value': [df_TTest]
    })
    resultdf_PM10 = pd.concat([resultdf_PM10,df_data])

In [ ]:
resultdf_PM10

In [ ]:
# PM25
for i in range(12):
    df_Pollutant = 'PM2.5'
    df_Month = i+1
    df_Average = myList_PM25_Clipped[i].pm2p5_conc.mean().data
    df_Variance = myList_PM25_Clipped[i].pm2p5_conc.std().data
    df_UP_Average = myList_PM25_Clipped_Up[i].pm2p5_conc.mean().data
    df_UP_Variance = myList_PM25_Clipped_Up[i].pm2p5_conc.std().data

    df_cams = myList_PM25_Clipped[i].pm2p5_conc.data
    df_cams = df_cams[~np.isnan(df_cams)]
    df_cams = df_cams.reshape(-1)
    
    df_camsup = myList_PM25_Clipped_Up[i].pm2p5_conc.data
    df_camsup = df_camsup[~np.isnan(df_camsup)]
    df_camsup = df_camsup.reshape(-1)

    df_FTest = f_test(df_cams,df_camsup)[1]
    df_TTest = ttest_ind(df_cams,df_camsup)[1]

    df_data = pd.DataFrame({
        'Pollutant': [df_Pollutant], 
        'Month': [df_Month],
        'Average': [df_Average],
        'Variance': [df_Variance],
        'Upscale Average': [df_UP_Average],
        'Upscale Variance': [df_UP_Variance],
        'F-Test p Value': [df_FTest],
        'T-Test p Value': [df_TTest]
    })
    resultdf_PM25 = pd.concat([resultdf_PM25,df_data])

In [ ]:
resultdf_PM25

In [ ]:
# The numbers are all stored as object in the DataFrame. Conver them to the float with a precision of 6. 
# Create a dict for faster loop
myDict = {
    "NO2": resultdf_NO2,
    "O3": resultdf_O3,
    "PM10": resultdf_PM10,
    "PM25": resultdf_PM25
}
myDict

In [ ]:
for df in myDict.values():
    df['Average'] = df['Average'].astype(float).round(decimals=6)
    df['Variance'] = df['Variance'].astype(float).round(decimals=6)
    df['Upscale Average'] = df['Upscale Average'].astype(float).round(decimals=6)
    df['Upscale Variance'] = df['Upscale Variance'].astype(float).round(decimals=6)
    df['F-Test p Value'] = df['F-Test p Value'].astype(float).round(decimals=6)
    df['T-Test p Value'] = df['T-Test p Value'].astype(float).round(decimals=6)

In [ ]:
resultdf_PM25.dtypes

In [ ]:
# Automatically export to excel file. Make sure the file is not open when overwriting existing one. 
# with pd.ExcelWriter('CAMS Upscale Model Tests.xlsx') as writer: 
#     resultdf_NO2.to_excel(writer, sheet_name='NO2', index=False)
#     resultdf_O3.to_excel(writer, sheet_name='O3', index=False)
#     resultdf_PM10.to_excel(writer, sheet_name='PM10', index=False)
#     resultdf_PM25.to_excel(writer, sheet_name='PM2.5', index=False)

In [ ]:
# Write to a single sheet for all the values. 
resultdf_combined_list = [resultdf_NO2, resultdf_O3, resultdf_PM25, resultdf_PM10]
resultdf_combined = pd.concat(resultdf_combined_list)
print(resultdf_combined)
resultdf_combined.to_excel('CAMS Upscale Model Tests - AIO.xlsx', index=False)